In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                status = ' '.join(html.xpath('//div[@class="product-basic-details__text--part"]/span/text()')).replace(input_.loc[a, 'Sku'].replace('# ', '#'), '').replace('(', '').replace(')', '').strip()
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//div[@class="product-basic-details__text--title"]/h3/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                summary = ' '.join(html.xpath('//div[@class="product-basic-details__text--summary"]/div/text()')).strip()
                
                # = = = = = = = = = = = = = = =
                                
                vehicle = ''.join(html.xpath('//div[@class="product-basic-details__applicable-car"]/text()')).strip().replace('; ', '\n')

                # = = = = = = = = = = = = = = =
                
                list_tr_1 = html.xpath('//table[@class="product-basic-details__table"]/tr')
                
                # = = = = = = = = = = = = = = =
                
                list_tr_2 = html.xpath('//div[@class="product-basic-details__text--section"]/table/tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Sku': input_.loc[a, 'Sku'],
                                         'Status': status,
                                         'Title': title,
                                         'Summary': summary,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': input_.loc[a, 'Src']}])

                for tr in list_tr_1:
                    if len(tr.xpath('./td/a')) == 0:
                        df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip()[:-1].upper()] = tr.xpath('./td[2]/text()')[0].strip()

                for tr in list_tr_2:
                    df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip().lower()] = tr.xpath('./td[2]/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Sku': input_.loc[a, 'Sku'],
                                     'Url': input_.loc[a, 'Url'],
                                     'Src': input_.loc[a, 'Src']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3575

[ok] - https://www.centricparts.com/disc-brake-pad-6828509
[尝试次数：1] - [剩余数量：3555] - [当前时间：10:09:11]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6828382
[尝试次数：1] - [剩余数量：3554] - [当前时间：10:09:11]

[ok] - https://www.centricparts.com/disc-brake-pad-6828530
[尝试次数：1] - [剩余数量：3553] - [当前时间：10:09:11]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6828403
[尝试次数：1] - [剩余数量：3552] - [当前时间：10:09:11]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6828546
[尝试次数：1] - [剩余数量：3551] - [当前时间：10:09:11]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6828558
[尝试次数：1] - [剩余数量：3550] - [当前时间：10:09:11]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6828547
[尝试次数：1] - [剩余数量：3549] - [当前时间：10:09:12]

[ok] - https://www.centricparts.com/disc-brake-pad-6828515
[尝试次数：1] - [剩余数量：3548] - [当前时间：10:09:12]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6828676
[尝试次数：1] - [剩余数量：3547] - [当前时间：10:09:12]

[ok] - https://www.centricparts.com/disc-brake-pad-set-68

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833165
[尝试次数：5] - [剩余数量：3476] - [当前时间：10:09:33]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832845
[尝试次数：5] - [剩余数量：3475] - [当前时间：10:09:34]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832502
[尝试次数：6] - [剩余数量：3474] - [当前时间：10:09:34]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832863
[尝试次数：5] - [剩余数量：3473] - [当前时间：10:09:34]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832954
[尝试次数：8] - [剩余数量：3472] - [当前时间：10:09:34]

[ok] - https://www.centricparts.com/disc-brake-pad-6832548
[尝试次数：8] - [剩余数量：3471] - [当前时间：10:09:35]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833026
[尝试次数：10] - [剩余数量：3470] - [当前时间：10:09:35]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832596
[尝试次数：4] - [剩余数量：3469] - [当前时间：10:09:36]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833428
[尝试次数：19] - [剩余数量：3468] - [当前时间：10:09:36]

[ok] - https://www.centricparts.com/disc-brake-pad-set-68

[ok] - https://www.centricparts.com/disc-brake-pad-6832479
[尝试次数：12] - [剩余数量：3397] - [当前时间：10:09:58]

[ok] - https://www.centricparts.com/disc-brake-pad-6832720
[尝试次数：2] - [剩余数量：3396] - [当前时间：10:09:59]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832842
[尝试次数：1] - [剩余数量：3395] - [当前时间：10:09:59]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832906
[尝试次数：2] - [剩余数量：3394] - [当前时间：10:10:00]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832665
[尝试次数：11] - [剩余数量：3393] - [当前时间：10:10:00]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832488
[尝试次数：6] - [剩余数量：3392] - [当前时间：10:10:01]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833371
[尝试次数：1] - [剩余数量：3391] - [当前时间：10:10:01]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833366
[尝试次数：1] - [剩余数量：3390] - [当前时间：10:10:01]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832549
[尝试次数：4] - [剩余数量：3389] - [当前时间：10:10:01]

[ok] - https://www.centricparts.com/disc-brake-pad-set-683271

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833070
[尝试次数：5] - [剩余数量：3318] - [当前时间：10:10:25]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832893
[尝试次数：3] - [剩余数量：3317] - [当前时间：10:10:25]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832712
[尝试次数：2] - [剩余数量：3316] - [当前时间：10:10:26]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833208
[尝试次数：4] - [剩余数量：3315] - [当前时间：10:10:26]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832692
[尝试次数：7] - [剩余数量：3314] - [当前时间：10:10:26]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833113
[尝试次数：4] - [剩余数量：3313] - [当前时间：10:10:26]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6832971
[尝试次数：6] - [剩余数量：3312] - [当前时间：10:10:26]

[ok] - https://www.centricparts.com/disc-brake-pad-6833246
[尝试次数：3] - [剩余数量：3311] - [当前时间：10:10:26]

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833043
[尝试次数：10] - [剩余数量：3310] - [当前时间：10:10:28]

[ok] - https://www.centricparts.com/disc-brake-pad-set-683

KeyboardInterrupt
2024-04-01T02:10:39Z


KeyboardInterrupt: 

[ok] - https://www.centricparts.com/disc-brake-pad-set-6833210
[尝试次数：7] - [剩余数量：3275] - [当前时间：10:10:39]

[ok] - https://www.centricparts.com/disc-brake-pad-6833368
[尝试次数：6] - [剩余数量：3274] - [当前时间：10:10:39]

[ok] - https://www.centricparts.com/disc-brake-pad-6832562
[尝试次数：3] - [剩余数量：3273] - [当前时间：10:10:39]

